# 🔍 FAISS Similarity Search with Sentence Transformers
This notebook loads academic papers from arXiv (PDFs), extracts text, chunks and embeds it using SentenceTransformers, and performs similarity search using FAISS.

In [1]:
!pip install sentence-transformers pymupdf faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [2]:
import requests
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss


In [3]:
# --- Step 1: Download PDFs from arXiv ---
pdf_urls = [
    "https://arxiv.org/pdf/2401.15884",
    "https://arxiv.org/pdf/2005.11401"
]

local_paths = []
for i, url in enumerate(pdf_urls):
    response = requests.get(url)
    filename = f"paper_{i}.pdf"
    with open(filename, "wb") as f:
        f.write(response.content)
    local_paths.append(filename)

print(f"Downloaded {len(local_paths)} PDFs.")


Downloaded 2 PDFs.


In [4]:
# --- Step 2: Extract text using PyMuPDF ---
def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

texts = [extract_text_from_pdf(path) for path in local_paths]
full_text = "\n".join(texts)
print(f"Total extracted characters: {len(full_text)}")


Total extracted characters: 130355


In [5]:
# --- Step 3: Split text into overlapping chunks ---
def split_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = split_text(full_text)
print(f"Generated {len(chunks)} chunks.")


Generated 326 chunks.


In [6]:
# --- Step 4: Generate embeddings using SentenceTransformer ---
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks, convert_to_numpy=True, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [7]:
# --- Step 5: Build FAISS index ---
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print(f"FAISS index built with {index.ntotal} vectors.")


FAISS index built with 326 vectors.


In [8]:
# --- Step 6: Similarity search interface ---
def search(query, k=3):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    print(f"\nTop {k} results for: '{query}'\n")
    for i, idx in enumerate(indices[0]):
        print(f"Result {i+1} (score={distances[0][i]:.2f}):\n{chunks[idx][:500]}\n{'-'*80}")


In [9]:
# --- Step 7: Try some sample questions based on the paper ---
sample_questions = [
    "What is RAG and how does it work?",
    "What is the difference between RAG-Sequence and RAG-Token?",
    "How does RAG use non-parametric memory?",
    "What tasks were used to evaluate RAG?",
    "How is Dense Passage Retrieval (DPR) used in RAG?",
    "What is the advantage of hybrid models over purely parametric models?",
    "What decoding strategies are used in RAG?",
    "How does RAG compare to T5 and BART?",
    "What datasets were used to benchmark RAG models?",
    "Can RAG models be updated without retraining?"
]

for q in sample_questions:
    search(q, k=2)



Top 2 results for: 'What is RAG and how does it work?'

Result 1 (score=0.80):
etability. RAG could be
employed in a wide variety of scenarios with direct beneﬁt to society, for example by endowing it
with a medical index and asking it open-domain questions on that topic, or by helping people be more
effective at their jobs.
With these advantages also come potential downsides: Wikipedia, or any potential external knowledge
source, will probably never be entirely factual and completely devoid of bias. Since RAG can be
employed as a language model, similar concerns as for GP
--------------------------------------------------------------------------------
Result 2 (score=0.90):
raphy (Min et al., 2023), Pub Health (Zhang et al.,
2023a), and Arc-Challenge (Bhakthavatsalam et al.,
2021) show that CRAG can significantly improve
the performance of standard RAG and state-of-the-
art Self-RAG, demonstrating its generalizability
across both short- and long-form generation tasks.
To facilitate o